In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import pandas as pd
import argparse                 
import sklearn 
import numpy as np 
import logging 
import sklearn.metrics as metrics 
import scikitplot as skplt
import matplotlib.pyplot as plt                                  
from argparse import ArgumentParser
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.utils.data import DataLoader, TensorDataset, Subset, Dataset, random_split
import torch.nn.functional as F
from collections import Counter, defaultdict
import itertools
import os 
import copy
import warnings
from dataload import makedata
from classifier import ClassifierTrainer
from models import VQ_Classifier
import itertools
from data import load_data
warnings.filterwarnings("ignore", category=UserWarning)
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
torch.set_num_threads(32) 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(911)

/home/smjo/anaconda3/envs/xai_timeseries/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
os.environ["CUDA_VISIBLE_DEVICES"] ='3'
len_position=13
dataset = 'mitbih'
classifier ='cnn'
classification_model =  '/home/smjo/xai_timeseries/vqvae/saved_models/classification/mitbih/1/cnn.pt'
vqvae_model = '/home/smjo/xai_timeseries/vqvae/saved_models/hard_mitbih/1/model_280.pt'
batch_size=256
ds = load_data(dataset, task='classification')


task is classification
dataset is mitbih
X shape:torch.Size([8039, 192]), y shape:torch.Size([8039, 2])


In [10]:
ds[0][0].shape

torch.Size([192])

In [18]:
train_size = int(0.8 * len(ds))
val_size = int(0.1 * len(ds))
test_size = len(ds) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(ds, [train_size, val_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False, pin_memory=True)

In [19]:
#Find masking token
end_tokens={}

conv_net = VQ_Classifier(
    num_classes = 2,
    vqvae_model = vqvae_model,
    positions =0,
    mask = 0,
    auc_classification = False,
    model_type = classifier
).to(device)

a = torch.load(classification_model)
conv_net.load_state_dict(a['model_state_dict'])

for param in conv_net.parameters():
    param.requires_grad = False

conv_net.eval()

VQ_Classifier(
  (vae): Residual_VQVAE(
    (vq): ResidualVQ(
      (layers): ModuleList(
        (0): VectorQuantize(
          (project_in): Identity()
          (project_out): Identity()
          (_codebook): EuclideanCodebook()
        )
      )
    )
    (encoder): Sequential(
      (0): Sequential(
        (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
        (1): ReLU()
      )
      (1): Sequential(
        (0): Conv1d(64, 64, kernel_size=(2,), stride=(2,))
        (1): ReLU()
      )
      (2): Sequential(
        (0): Conv1d(64, 64, kernel_size=(2,), stride=(2,))
        (1): ReLU()
      )
      (3): Sequential(
        (0): Conv1d(64, 64, kernel_size=(2,), stride=(2,))
        (1): ReLU()
      )
      (4): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
    )
    (decoder): Sequential(
      (0): Sequential(
        (0): ConvTranspose1d(64, 64, kernel_size=(2,), stride=(2,))
        (1): ReLU()
      )
      (1): Sequential(
        (0): ConvTranspose1d(64, 64, kernel_s

In [20]:

masked_regions = conv_net.mask_region(ds.data[:64, :].to(device))
masked_regions

[tensor(117, device='cuda:0')]

In [21]:
#Make position answer
answer = {}         
arr = [i for i in range(len_position)]
roc_auc_scores=[]


for cnt in range(len_position):
    max_auc=-1
    print("arr:", arr)
    for i in arr:
        #ECG Dataset
        positions = list(answer.values())+[i]
        net = VQ_Classifier(
            num_classes = 2,
            vqvae_model = vqvae_model,
            positions = positions,
            mask = masked_regions,
            auc_classification = True,
            model_type = classifier
        ).to(device)
        
        
        net.load_state_dict(torch.load(classification_model)["model_state_dict"])
        net = net.to(device)
        for param in net.parameters():
            param.requires_grad = False        
        net.eval()   

        with torch.no_grad():    
            score=0
            for idx, (data, labels) in enumerate(test_loader):
                data = data.unsqueeze(1).float()
                labels = labels.type(torch.LongTensor)
                labels = torch.argmax(labels, dim=1)
                data, labels = data.to(device), labels.to(device)
                output, codebook_tokens, recon, input= net(data)
                           
                score += sklearn.metrics.roc_auc_score(labels.cpu(), output[:,1].cpu())
                break
        score = score / (idx+1)
        if score > max_auc:
            answer[f"top_{cnt}"]=i
            max_auc = copy.deepcopy(score)
    roc_auc_scores.append(max_auc)        
    arr.remove(answer[f"top_{cnt}"])
    

arr: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


100%|██████████| 256/256 [00:00<00:00, 943.38it/s]


arr: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


100%|██████████| 256/256 [00:00<00:00, 512.71it/s]


arr: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


...

100%|██████████| 256/256 [00:02<00:00, 101.47it/s]


arr: [11, 12]


100%|██████████| 256/256 [00:02<00:00, 93.59it/s]


arr: [12]


100%|██████████| 256/256 [00:02<00:00, 93.45it/s]


In [22]:
answer.values()

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

In [ ]:
import pickle
with open("answer.txt", "wb") as fp:
    pickle.dump(answer.values(), fp)

In [ ]:
print(answer)

In [ ]:
python position_answer.py --dataset mitbih --model_type cnn --vqvae_model